# Efficient pathwise posterior sample optimization - Emittance

In [1]:
import torch
import gpytorch
import botorch
from matplotlib import pyplot as plt

from emitutils import (toy_beam_size_squared_nd, fit_gp_model_emittance, sum_samplewise_emittance_flat_X_wrapper_for_scipy, 
                       post_emit, sum_samplewise_emittance_flat_X_wrapper_for_torch)

from samplingutils import draw_product_kernel_post_paths
from utils import unif_random_sample_domain
from utils import build_mesh_domain

In [2]:

domain = torch.tensor([[-2,2], [-35, 5]]).double()
ndim = domain.shape[0]
# train_x = build_mesh_domain(3, domain)[0]
# train_x = unif_random_sample_domain(15, domain).double()
train_x = unif_random_sample_domain(35, domain).double()
train_y = toy_beam_size_squared_nd(train_x).double()*1e6



model = fit_gp_model_emittance(train_x, train_y)


n_samples = 5
post_paths = draw_product_kernel_post_paths(model, n_samples=n_samples)




C:\Users\Dylan\SLAC\botorch-bax\botorch\models\gpytorch.py:113: UserWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  warnings.warn(


In [3]:
X_meas = torch.linspace(*domain[-1], 4)


In [4]:
x_inits = unif_random_sample_domain(n_samples, domain[:-1]).double().requires_grad_(True)
x_inits

tensor([[1.1287],
        [0.5554],
        [0.5447],
        [1.7603],
        [1.5044]], dtype=torch.float64, requires_grad=True)

In [5]:
sum(x_inits.size())

6

In [6]:
from scipy.optimize import minimize

target_func_for_scipy = sum_samplewise_emittance_flat_X_wrapper_for_scipy(post_paths, n_samples, X_meas)


################################
#with jacobian from torch (NOTE: backward doesn't work for the target function due to shape problems!)

target_func_for_torch = sum_samplewise_emittance_flat_X_wrapper_for_torch(post_paths, n_samples, X_meas)

def wrap_jac_for_scipy(target_func_for_torch):
    def wrapped_func(x):
        return torch.autograd.functional.jacobian(target_func_for_torch, torch.tensor(x)).detach().numpy()
    return wrapped_func

target_func_jac_callable = wrap_jac_for_scipy(target_func_for_torch)

res = minimize(target_func_for_scipy, x_inits.flatten().detach().numpy(), bounds = domain[:-1].repeat(n_samples,1), 
               jac = target_func_jac_callable, tol=1e-6, options = {'eps': 1e-03})
# ################################
# #without jacobian from torch

# res = minimize(target_func_for_scipy, x_inits.flatten().detach().numpy(), bounds = domain[:-1].repeat(n_samples,1), 
#                tol=1e-6, options = {'eps': 1e-03})

# # ################################

x_stars_flat = res.x

x_stars = x_stars_flat.reshape(n_samples,-1)

tensor([-35.0000, -21.6667,  -8.3333,   5.0000])
tensor([[30.4399,  5.7405,  5.7403, 30.4395],
        [17.4045,  3.4577,  3.3752, 17.1571],
        [17.2960,  3.3566,  3.3163, 17.1750],
        [51.8338,  9.1109,  9.8733, 54.1209],
        [43.1965,  8.0216,  8.1606, 43.6136]], dtype=torch.float64,
       grad_fn=<SliceBackward0>)
tensor([-35.0000, -21.6667,  -8.3333,   5.0000])
tensor([[30.4399,  5.7405,  5.7403, 30.4395],
        [17.4045,  3.4577,  3.3752, 17.1571],
        [17.2960,  3.3566,  3.3163, 17.1750],
        [51.8338,  9.1109,  9.8733, 54.1209],
        [43.1965,  8.0216,  8.1606, 43.6136]], dtype=torch.float64,
       grad_fn=<SliceBackward0>)


RuntimeError: Expected size for first two dimensions of batch2 tensor to be: [5, 3] but got: [5, 5].

In [7]:
target_func_for_torch(x_inits.flatten()).backward()

tensor([-35.0000, -21.6667,  -8.3333,   5.0000])
tensor([[30.4399,  5.7405,  5.7403, 30.4395],
        [17.4045,  3.4577,  3.3752, 17.1571],
        [17.2960,  3.3566,  3.3163, 17.1750],
        [51.8338,  9.1109,  9.8733, 54.1209],
        [43.1965,  8.0216,  8.1606, 43.6136]], dtype=torch.float64,
       grad_fn=<SliceBackward0>)


RuntimeError: Expected size for first two dimensions of batch2 tensor to be: [5, 3] but got: [5, 5].

In [17]:
from emitutils import compute_emits_from_batched_beamsize_scans

with torch.autograd.detect_anomaly():
    xs_meas_test = torch.tensor([-35.0000, -21.6667,  -8.3333,   5.0000]).requires_grad_(True)
    ys_batch_test = torch.tensor([[30.4399,  5.7405,  5.7403, 30.4395],
            [17.4045,  3.4577,  3.3752, 17.1571],
            [17.2960,  3.3566,  3.3163, 17.1750],
            [51.8338,  9.1109,  9.8733, 54.1209],
            [43.1965,  8.0216,  8.1606, 43.6136]] ).requires_grad_(True)

    emits_sq = compute_emits_from_batched_beamsize_scans(xs_meas_test, ys_batch_test)[1]
    emits_sq.sum().backward()

tensor([-35.0000, -21.6667,  -8.3333,   5.0000], requires_grad=True)
tensor([[30.4399,  5.7405,  5.7403, 30.4395],
        [17.4045,  3.4577,  3.3752, 17.1571],
        [17.2960,  3.3566,  3.3163, 17.1750],
        [51.8338,  9.1109,  9.8733, 54.1209],
        [43.1965,  8.0216,  8.1606, 43.6136]], requires_grad=True)


C:\Users\Dylan\AppData\Local\Temp\ipykernel_9076\2861227258.py:3: UserWarning: Anomaly Detection has been enabled. This mode will increase the runtime and should only be enabled for debugging.
  with torch.autograd.detect_anomaly():
C:\Users\Dylan\Anaconda3\envs\BOTORCH\lib\site-packages\torch\autograd\__init__.py:173: UserWarning: Error detected in LinalgLstsqBackward0. Traceback of forward call that caused the error:
  File "C:\Users\Dylan\Anaconda3\envs\BOTORCH\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\Dylan\Anaconda3\envs\BOTORCH\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\Dylan\Anaconda3\envs\BOTORCH\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\Dylan\Anaconda3\envs\BOTORCH\lib\site-packages\traitlets\config\application.py", line 982, in launch_instance
    app.start()
  File "C:\Users\Dylan\Anaconda3\envs\BOTOR

RuntimeError: mat1 and mat2 shapes cannot be multiplied (5x3 and 4x5)

In [ ]:
# from torchmin import minimize

# target_func_for_torch = sum_samplewise_emittance_flat_X_wrapper_for_torch(post_paths, n_samples, X_meas)
# res = minimize(target_func_for_torch, x_inits.flatten(), method = 'bfgs', tol=1e-6)


# x_stars_flat = res.x

# x_stars = x_stars_flat.reshape(n_samples,-1)

In [ ]:
# from autograd_minimize import minimize

# X_tuning_init = x_inits.flatten().detach().numpy()

# target_func_for_torch = sum_samplewise_emittance_flat_X_wrapper_for_torch(post_paths, n_samples, X_meas)
# res = minimize(target_func_for_torch, X_tuning_init, backend='torch', tol=1e-6)


# x_stars_flat = res.x

# x_stars = x_stars_flat.reshape(n_samples,-1)

In [ ]:
x_stars

In [ ]:
sid = 0
fig, ax = plt.subplots(1)

nsteps_mesh = 100
xs_mesh_n_by_d, x_mesh_tuple = build_mesh_domain(nsteps_mesh, domain)
x0mesh, x1mesh = x_mesh_tuple
ys_mesh = post_paths(xs_mesh_n_by_d).reshape(n_samples,nsteps_mesh,nsteps_mesh)
ax.pcolor(x0mesh.detach().numpy(), x1mesh.detach().numpy(), ys_mesh[sid].detach().numpy())
# ax.scatter(*xs_star[sid], marker='x', s=120, color='r', label='scipy min')

ax.set_title('Pathwise Posterior Sample Optimization')
ax.set_ylabel('$x_2$')
ax.set_xlabel('$x_1$')
ax.legend()

textstr = 'sample ' + str(sid)


# these are matplotlib.patch.Patch properties
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

# place a text box in upper left in axes coords
ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=14,
        verticalalignment='top', horizontalalignment = 'left', bbox=props)

In [ ]:
X_tuning_scan = torch.linspace(*domain[0],100)
X_tuning_scan = X_tuning_scan.reshape(-1,1)
emits = post_emit(post_paths, X_tuning_scan, X_meas, samplewise=False, squared=True)

In [ ]:
emit_stars = post_emit(post_paths, torch.tensor(x_stars), X_meas, samplewise=True, squared=True)

p = plt.plot(X_tuning_scan.flatten(), emits[0].detach().numpy(), label='post. sample')
x_star = x_stars[0]
emit_star = emit_stars[0]
plt.scatter([x_star[0]],[emit_star.detach().numpy()], c=p[-1].get_color(), label='scipy min')

for i in range(1, n_samples):
    p = plt.plot(X_tuning_scan.flatten(), emits[i].detach().numpy())
    x_star = x_stars[i]
    emit_star = emit_stars[i]
    plt.scatter([x_star[0]],[emit_star.detach().numpy()], c=p[-1].get_color())

plt.title('Pathwise Sample $\epsilon^2$ Optimization - 2d')
plt.xlabel('$x_1$')
plt.ylabel('$\epsilon^{2}$', fontsize=14)
plt.legend()

In [ ]:
from algorithms import ScipyMinimizeEmittance, GridMinimizeEmittance

In [ ]:
algo = ScipyMinimizeEmittance(domain=domain, n_samples=5)
# algo = GridMinimizeEmittance(domain=domain, n_samples=5, n_steps_tuning_params=10, squared=True)

In [ ]:
xs_exe, ys_exe = algo.get_exe_paths(model)

In [ ]:
xs_exe.shape

In [ ]:
ys_exe.shape

In [ ]:
xs_exe

In [ ]:
ys_exe

In [ ]:
from emitutils import EmittanceModule
from botorch.generation.gen import gen_candidates_scipy
from botorch.optim.initializers import gen_batch_initial_conditions

In [ ]:
#need to wrap the function for the emittance module forward pass so that it can handle a batch of flat tensor inputs

emitmodule = EmittanceModule(post_paths, n_samples, X_meas)
bounds = domain[:-1].repeat(n_samples,1).T
Xinit = gen_batch_initial_conditions(
        emitmodule, bounds, q=1, num_restarts=25, raw_samples=500
 )
batch_candidates, batch_acq_values = gen_candidates_scipy(
        initial_conditions=Xinit,
        acquisition_function=emitmodule,
        lower_bounds=bounds[0],
        upper_bounds=bounds[1],
            )